## Talk is cheap, lets make a database and query it.  
### For this presentation on my experience in SQL; I'll be taking a dataset I made by hydrating tweets from the 2016 presidential election and scraping tweets from their retweet sources to make the beginnings of an extendable database.
#### I'll be combining 6k tweets from Hillary and Trump with approx. 100 tweets from every source they've retweeted from in those original 6k.


In [257]:
import os
import json
import itertools
import ast
import pandas as pd
import sqlite3 
import sqlalchemy
from sqlalchemy import create_engine

In [318]:
hill_trump = pd.read_csv('https://raw.githubusercontent.com/xkong100/Data620_Team3/master/final_Project/tweets.csv')
hill_trump.head(3)

,id,handle,text,is_retweet,original_author,time,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,...,place_type,place_country_code,place_country,place_contained_within,place_attributes,place_bounding_box,source_url,truncated,entities,extended_entities
0,780925634159796224,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'media': [{'display_url': 'pic.twitter.com/Xr...
1,780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27T23:45:00,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,http://twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/t0...,{'media': [{'display_url': 'pic.twitter.com/t0...
2,780911564857761793,HillaryClinton,Couldn't be more proud of @HillaryClinton. Her...,True,POTUS,2016-09-27T23:26:40,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://about.twitter.com/products/tweetdeck,False,"{'user_mentions': [{'id_str': '1536791610', 'n...",NaN


In [217]:
#To get an idea of the original twitter database schema, we look for IDs
unique_ids = [col for col in hill_trump.columns if 'id' in col]
unique_ids

['id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'place_id']

In [218]:
# Look like we can find retweeter IDs in the entities JSON
hill_trump.iloc[2]['entities']

"{'user_mentions': [{'id_str': '1536791610', 'name': 'President Obama', 'id': 1536791610, 'screen_name': 'POTUS', 'indices': [3, 9]}, {'id_str': '1339835893', 'name': 'Hillary Clinton', 'id': 1339835893, 'screen_name': 'HillaryClinton', 'indices': [37, 52]}], 'symbols': [], 'urls': [], 'hashtags': []}"

### Since the purpose of my database is to be used for basic tweet analysis using an ORM library, I want to keep it minimalistic and clean. Lets remove unneeded fields. 


In [18]:
print(hill_trump.isnull().sum())

id                            0
handle                        0
text                          0
is_retweet                    0
original_author            5722
time                          0
in_reply_to_screen_name    6236
in_reply_to_status_id      6242
in_reply_to_user_id        6236
is_quote_status               0
lang                          0
retweet_count                 0
favorite_count                0
longitude                  6432
latitude                   6432
place_id                   6240
place_full_name            6240
place_name                 6240
place_type                 6240
place_country_code         6240
place_country              6240
place_contained_within     6240
place_attributes           6240
place_bounding_box         6240
source_url                    0
truncated                     0
entities                      0
extended_entities          5096
dtype: int64


In [319]:
keep = ['id','handle','text','is_retweet','original_author','time',
        'is_quote_status','lang','retweet_count','favorite_count',
        'source_url','truncated','entities']
hill_trump = hill_trump[keep]
print(hill_trump.isnull().sum())


id                    0
handle                0
text                  0
is_retweet            0
original_author    5722
time                  0
is_quote_status       0
lang                  0
retweet_count         0
favorite_count        0
source_url            0
truncated             0
entities              0
dtype: int64


In [320]:
rt = hill_trump[hill_trump['is_retweet'] == True]
userID = {}
for index, row in rt.iterrows():
    obj = ast.literal_eval(row['entities'])
    for user in obj['user_mentions']:
        userID[user['screen_name']] = user['id']

#manual add Hill n Trump
userID['realDonaldTrump'] = 25073877
userID['HillaryClinton'] = 1339835893


### Next dataset; the scraped tweets by other political sources


In [321]:
retweet_source_path = 'C:/Users/Exped/Desktop/SQL/retweets/'
files_to_read = os.listdir(retweet_source_path)
files_to_read[10:18]

['Austin Hunt.csv',
 'Barack Obama.csv',
 'Barbara Boxer.csv',
 'Barbara Kinney.csv',
 'Ben Kesling.csv',
 'Ben Shapiro.csv',
 'Ben Smith.csv',
 'Ben White.csv']

In [322]:
retweet_sources = pd.DataFrame()
for filename in files_to_read:
    try:
        exp = pd.read_csv(open(retweet_source_path+filename,'rb'),index_col=False,sep=';',names=['timestamp','sn','text'],header=0)
        retweet_sources = pd.concat([retweet_sources, exp], ignore_index=True)
    except Exception as e: 
        print(filename)
        print(e)

Constance Zimmer.csv
Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

retweet_origin.csv
Too many columns specified: expected 3 and found 1


In [236]:
retweet_sources['sn'].unique()

array(['AdrianaCohen16', 'algore', 'AmandaRenteria', 'AndraDayMusic',
       'AndrewJenks', 'AustinHunt', 'BarackObama', 'BarbaraBoxer',
       'barb_kinney', 'bkesling', 'SenSanders', 'billclinton',
       'BishopGarrison', 'BretEastonEllis', 'brianefallon',
       'cameronesposito', 'Carl_C_Icahn', 'ChadHGriffin', 'ChelseaClinton',
       'ChrisMurphyCT', 'clairecmc', 'ClintonFdn', 'CNBCnow',
       'CNBCopinion', 'CollegeDems', 'RepTimRyan', 'CLewandowski_',
       'CWAUnion', 'DanMalloyCT', 'DanScavino', 'DarrenJJordan',
       'dmartosko', 'DemConvention', 'ddlovato', 'OhioPoliticsNow',
       'dominiclowell', 'realDonaldTrump', 'DonaldJTrumpJr',
       'RealBenCarson', 'robertjeffress', 'EconomicPolicy',
       'ElizabethBanks', nan, 'feministabulous', 'SenWarren',
       'StopBigMoney', 'EricTrump', 'ErinBurnett', 'Evan_McMullin',
       'EveryVoice', 'Everytown', 'ezraklein', 'FieldofFight',
       'funnyordie', 'GabbyGiffords', 'GavinNewsom', 'GeorgeTakei',
       'GeraldoRive

In [323]:
retweet_sources.columns = ['timestamp','screen_name','text']

In [324]:
retweet_sources['id'] = retweet_sources['screen_name'].map(userID)
retweet_sources['timestamp'] = pd.to_datetime(retweet_sources['timestamp'],errors='coerce')
retweet_sources = retweet_sources.dropna(subset=['timestamp'])
hill_trump['timestamp'] = pd.to_datetime(hill_trump['time'])
hill_trump.head(4)

,id,handle,text,is_retweet,original_author,time,is_quote_status,lang,retweet_count,favorite_count,source_url,truncated,entities,timestamp
0,780925634159796224,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,False,en,218,651,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Xr...,2016-09-28 00:22:34
1,780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27T23:45:00,False,en,2445,5308,http://twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/t0...,2016-09-27 23:45:00
2,780911564857761793,HillaryClinton,Couldn't be more proud of @HillaryClinton. Her...,True,POTUS,2016-09-27T23:26:40,False,en,7834,27234,https://about.twitter.com/products/tweetdeck,False,"{'user_mentions': [{'id_str': '1536791610', 'n...",2016-09-27 23:26:40
3,780907038650068994,HillaryClinton,"If we stand together, there's nothing we can't...",False,NaN,2016-09-27T23:08:41,False,en,916,2542,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Q3...,2016-09-27 23:08:41


In [325]:
hill_trump.columns = ['tweet_id','screen_name','tweet','is_retweet','original_author','garbage','is_quote_status','lang','retweet_count','favorite_count','source_url','truncated','entities','timestamp']
hill_trump = hill_trump.drop(['garbage'], axis=1)

In [326]:
retweet_sources.columns = ['timestamp','screen_name','tweet','user_id']
retweet_sources.head(1)

,timestamp,screen_name,tweet,user_id
0,2017-02-01 03:26:33,AdrianaCohen16,@CNNPolitics Said none of the millions of Amer...,1.628841e+09


In [327]:
tweets = pd.concat([hill_trump,retweet_sources], axis=0, ignore_index=True)
tweets['user_id'] = tweets['screen_name'].map(userID)

In [328]:
tweets.head(4)

,entities,favorite_count,is_quote_status,is_retweet,lang,original_author,retweet_count,screen_name,source_url,timestamp,truncated,tweet,tweet_id,user_id
0,{'media': [{'display_url': 'pic.twitter.com/Xr...,651.0,False,False,en,NaN,218.0,HillaryClinton,https://studio.twitter.com,2016-09-28 00:22:34,False,The question in this election: Who can put the...,7.809256e+17,1339835893
1,{'media': [{'display_url': 'pic.twitter.com/t0...,5308.0,False,True,en,timkaine,2445.0,HillaryClinton,http://twitter.com,2016-09-27 23:45:00,False,"Last night, Donald Trump said not paying taxes...",7.809162e+17,1339835893
2,"{'user_mentions': [{'id_str': '1536791610', 'n...",27234.0,False,True,en,POTUS,7834.0,HillaryClinton,https://about.twitter.com/products/tweetdeck,2016-09-27 23:26:40,False,Couldn't be more proud of @HillaryClinton. Her...,7.809116e+17,1339835893
3,{'media': [{'display_url': 'pic.twitter.com/Q3...,2542.0,False,False,en,NaN,916.0,HillaryClinton,https://studio.twitter.com,2016-09-27 23:08:41,False,"If we stand together, there's nothing we can't...",7.809070e+17,1339835893


In [329]:
users = tweets[['screen_name','user_id']]
tweets = tweets[['tweet_id','screen_name','tweet','is_retweet','original_author','timestamp']]

In [330]:
users = users.drop_duplicates()
users.columns

Index(['screen_name', 'user_id'], dtype='object')

In [331]:
tweets = tweets.drop_duplicates(subset='timestamp')
tweets.columns

Index(['tweet_id', 'screen_name', 'tweet', 'is_retweet', 'original_author',
       'timestamp'],
      dtype='object')

# SQL 




In [355]:
#Creates a connection to an existing database, or creates a new one if doesn't exist
conn = sqlite3.connect('SQLite/twitter.db')
print(sqlite3.version)


2.6.0


# We want to include all the tweets with minimal redundancy;
## We'll create two tables; one to hold users, the other to hold tweets
## Primary keys, user-ID.

In [356]:
c = conn.cursor()
c.execute("""CREATE TABLE IF NOT EXISTS users (
                    screen_name text NOT NULL PRIMARY KEY,
                    user_id integer NOT NULL)""")

c.execute("""CREATE TABLE IF NOT EXISTS tweets (
                    tweet_id integer,
                    screen_name text NOT NULL,
                    tweet text NOT NULL,
                    is_retweet integer,
                    original_author text,
                    timestamp integer,
                    FOREIGN KEY (screen_name) REFERENCES users);""")

In [357]:
engine = create_engine('sqlite:///SQLite/twitter.db')
tweets.to_sql('tweets',con=engine,if_exists='append',index=False)


In [358]:
users.to_sql('users',con=engine,if_exists='append',index=False)

# Lets compare SQL to Pandas

## The WHERE clause

## If I wanted to find all of Hillary's retweets, I'd use SELECT and WHERE

#### SELECT * (Give me all)
#### FROM tweets (The table known as 'tweets')
#### WHERE is_retweet = 1 (Only where retweet = 1 (True))
#### AND screen_name = 'HillaryClinton'

<img src="https://github.com/parastyle/SQL-Mini/blob/master/SQLite/1.PNG?raw=true">

## How would we do that in pandas?

### Create a subset based on the constraints

In [365]:
ex1 = tweets[(tweets['screen_name']=='HillaryClinton') & (tweets['is_retweet']==1)]
ex1.head(1)

,tweet_id,screen_name,tweet,is_retweet,original_author,timestamp
1,7.809162e+17,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27 23:45:00


# Find a frequent something; a customer or event

## I would use the GROUP and ORDER clauses to find frequency

## Say I wanted to find out who, was the 2nd most retweeted by Trump

#### SELECT original_author
#### FROM tweets
#### WHERE screen_name = 'realDonaldTrump'
#### AND is_retweet = 1
#### GROUP BY original_author
#### ORDER BY COUNT(*) DESC (Use the count function for frequency, have results descend)
#### LIMIT 2; (The name on the bottom of this list will be the second most frequently retweeted by Trump

<img src="https://github.com/parastyle/SQL-Mini/blob/master/SQLite/2.PNG?raw=true">

## How would we do that in pandas?

### Create a subset based on the constraints, but since pandas doesn't use simple english clauses, its good convention to do this out in a couple steps so people can interpret what you've done.  


In [379]:
step1 = tweets[(tweets['is_retweet']==True) & (tweets['screen_name']=='realDonaldTrump')]
step2 = step1['original_author'].value_counts()[:2]
step2

EricTrump        17
DRUDGE_REPORT     9
Name: original_author, dtype: int64